# 103: Advanced Feature Engineering

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** mathematical foundations of feature interactions, polynomial expansion, and encoding strategies
- **Implement** feature engineering techniques from scratch using NumPy and pandas
- **Build** production-ready feature pipelines with sklearn transformers
- **Apply** domain-specific feature engineering to semiconductor parametric test data
- **Evaluate** feature importance and engineering impact on model performance

## 📚 What is Feature Engineering?

Feature engineering is the process of creating new input variables (features) from existing raw data to improve machine learning model performance. It combines domain knowledge, mathematical transformations, and automated discovery methods to expose patterns that algorithms can exploit.

Unlike automated feature learning in deep learning, traditional ML models (linear regression, tree-based methods, SVMs) benefit enormously from well-engineered features. A carefully crafted feature can capture complex relationships that would require thousands of model parameters to learn automatically.

In post-silicon validation, feature engineering transforms raw parametric test measurements (voltage, current, frequency) into meaningful device characteristics (power consumption, thermal stress, efficiency metrics) that directly correlate with yield and reliability outcomes.

**Why Feature Engineering?**
- ✅ **Boost Model Performance**: Well-engineered features can improve R² from 0.70 to 0.95+ with same algorithm
- ✅ **Reduce Model Complexity**: Fewer, better features = faster training, simpler models, easier deployment
- ✅ **Encode Domain Knowledge**: Capture physics-based relationships (power = voltage × current) that data alone may not reveal
- ✅ **Handle Non-Linearity**: Polynomial and interaction terms allow linear models to fit curved decision boundaries
- ✅ **Manage Categoricals**: Advanced encoding (target, hash) handles high-cardinality variables efficiently

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: Power Consumption Prediction**
- **Input**: Vdd (supply voltage), Idd (supply current), frequency, temperature from parametric tests
- **Engineered Features**: Power (Vdd×Idd), power density, thermal coefficient, efficiency ratio
- **Output**: Predicted device power consumption with ±5% accuracy
- **Value**: Early detection of high-power outliers before customer shipment saves $50K-$500K per RMA

**Use Case 2: Wafer-Level Yield Modeling**
- **Input**: Die coordinates (x, y), test parameters, lot ID from wafer test STDF
- **Engineered Features**: Radial distance from center, edge proximity, spatial clusters, lot-level statistics
- **Output**: Spatial yield map identifying systematic defect patterns
- **Value**: Root cause localization (lithography, CMP, implant issues) reduces time-to-fix from weeks to days

**Use Case 3: Test Time Optimization**
- **Input**: 500+ parametric test measurements per device, test execution timestamps
- **Engineered Features**: Test correlations, redundancy scores, temporal patterns, equipment drift indicators
- **Output**: Pruned test suite maintaining 99.5% defect coverage at 40% reduced test time
- **Value**: $2M-$5M annual savings per product line from reduced ATE (Automated Test Equipment) hours

**Use Case 4: Bin Prediction (Pass/Fail Classification)**
- **Input**: Raw parametric measurements, environmental conditions, device metadata
- **Engineered Features**: Polynomial expansions capturing non-linear spec limits, interaction terms for correlated failures
- **Output**: Probabilistic bin prediction (BIN1=premium, BIN2=standard, BIN7=fail)
- **Value**: Adaptive test insertion based on early predictions reduces escapes 30-50%

## 🔄 Feature Engineering Workflow

```mermaid
graph TB
    A[Raw STDF Data] --> B[Domain Analysis]
    B --> C{Feature Type?}
    
    C -->|Numerical| D[Interactions & Polynomials]
    C -->|Categorical| E[Encoding Strategy]
    C -->|Spatial| F[Coordinate Transforms]
    C -->|Temporal| G[Time-Based Features]
    
    D --> H[Feature Selection]
    E --> H
    F --> H
    G --> H
    
    H --> I[Validate on Holdout]
    I --> J{Performance OK?}
    
    J -->|No| K[Refine Features]
    K --> C
    J -->|Yes| L[Production Pipeline]
    
    style A fill:#e1f5ff
    style L fill:#e1ffe1
    style I fill:#fff4e1
```

## 📊 Learning Path Context

**Prerequisites:**
- **010**: Linear Regression - Feature-target relationships
- **016**: Decision Trees - Feature importance concepts
- **022**: Random Forest - Ensemble feature selection
- **041**: Model Evaluation - Validation strategies

**This Notebook (103):**
- Feature interaction terms (multiplicative combinations)
- Polynomial feature expansion (degree 2, 3)
- Target encoding for categoricals (smoothed mean encoding)
- Feature hashing for high-cardinality variables
- Automated feature engineering concepts

**Next Steps:**
- **104**: Model Interpretability - Understand which engineered features matter (SHAP, LIME)
- **105**: AutoML & NAS - Automated feature discovery at scale
- **108**: Feature Stores - Production feature management with Feast

---

Let's engineer features that unlock hidden patterns in semiconductor test data! 🚀

## 1. Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction import FeatureHasher
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Random seed for reproducibility
np.random.seed(42)

print("✅ Environment ready for advanced feature engineering!")

## 2. Generate Semiconductor Test Data

In [ ]:
# Generate synthetic STDF-like parametric test data
n_devices = 1000

# Test parameters
vdd = np.random.normal(1.2, 0.05, n_devices)  # Supply voltage
idd = np.random.normal(50, 5, n_devices)      # Supply current (mA)
freq = np.random.normal(2000, 100, n_devices) # Frequency (MHz)
temp = np.random.normal(85, 10, n_devices)    # Temperature (°C)
vth = np.random.normal(0.4, 0.02, n_devices)  # Threshold voltage

# Wafer position
die_x = np.random.randint(0, 20, n_devices)
die_y = np.random.randint(0, 20, n_devices)

# Lot IDs (categorical)
lot_ids = np.random.choice(['LOT_A', 'LOT_B', 'LOT_C', 'LOT_D'], n_devices)

# Target: yield (influenced by interactions)
power = vdd * idd
thermal_stress = temp * freq / 1000
yield_score = 95 - 0.3 * power - 0.01 * thermal_stress + 2 * vth + np.random.normal(0, 2, n_devices)
yield_score = np.clip(yield_score, 70, 100)

# Create DataFrame
df = pd.DataFrame({
    'vdd': vdd,
    'idd': idd,
    'freq': freq,
    'temp': temp,
    'vth': vth,
    'die_x': die_x,
    'die_y': die_y,
    'lot_id': lot_ids,
    'yield': yield_score
})

print(f"Dataset: {df.shape[0]} devices, {df.shape[1]} columns")
print(f"\nFirst 3 rows:")
df.head(3)

## 3. Feature Interaction Terms (From Scratch)

**Concept:** Multiply features to capture their combined effect.

**Mathematics:**
$$f_{interaction}(x_i, x_j) = x_i \times x_j$$

**Why It Works:** Many real-world phenomena are multiplicative (e.g., power = voltage × current).

In [ ]:
# From-scratch implementation of feature interactions
def create_interaction_features(df, feature_pairs):
    """
    Create interaction features from specified pairs.
    
    Parameters:
    - df: DataFrame
    - feature_pairs: List of tuples (feature1, feature2)
    
    Returns:
    - DataFrame with new interaction columns
    """
    df_copy = df.copy()
    
    for feat1, feat2 in feature_pairs:
        interaction_name = f"{feat1}_x_{feat2}"
        df_copy[interaction_name] = df_copy[feat1] * df_copy[feat2]
        print(f"Created: {interaction_name}")
    
    return df_copy

# Define important interactions for semiconductor devices
interactions = [
    ('vdd', 'idd'),      # Power consumption
    ('temp', 'freq'),    # Thermal stress
    ('vdd', 'vth'),      # Overdrive voltage
    ('die_x', 'die_y'),  # Spatial correlation
]

df_interact = create_interaction_features(df, interactions)
print(f"\nNew shape: {df_interact.shape}")
print(f"New columns: {[col for col in df_interact.columns if '_x_' in col]}")

## 4. Polynomial Features (Production)

**Concept:** Create polynomial combinations of features (x², x³, x₁×x₂, etc.)

**sklearn Implementation:**

In [ ]:
# Select numerical features for polynomial expansion
num_features = ['vdd', 'idd', 'freq', 'temp', 'vth']
X_num = df[num_features]

# Create polynomial features (degree=2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_num)

# Get feature names
poly_feature_names = poly.get_feature_names_out(num_features)

# Create DataFrame
df_poly = pd.DataFrame(X_poly, columns=poly_feature_names)

print(f"Original features: {X_num.shape[1]}")
print(f"Polynomial features: {df_poly.shape[1]}")
print(f"\nSample polynomial features:")
print(poly_feature_names[:10])

# Visualize impact on model performance
y = df['yield']
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)
X_train_poly, X_test_poly, _, _ = train_test_split(df_poly, y, test_size=0.2, random_state=42)

# Model with original features
model_linear = Ridge(alpha=1.0)
model_linear.fit(X_train, y_train)
y_pred_linear = model_linear.predict(X_test)
r2_linear = r2_score(y_test, y_pred_linear)

# Model with polynomial features
model_poly = Ridge(alpha=1.0)
model_poly.fit(X_train_poly, y_train)
y_pred_poly = model_poly.predict(X_test_poly)
r2_poly = r2_score(y_test, y_pred_poly)

print(f"\nR² Score - Linear features: {r2_linear:.4f}")
print(f"R² Score - Polynomial features: {r2_poly:.4f}")
print(f"Improvement: {(r2_poly - r2_linear) * 100:.2f}%")

## 5. Target Encoding for Categorical Variables

**Concept:** Encode categories by their mean target value.

**Mathematics:**
$$\text{encoding}(c) = \frac{\sum_{i \in c} y_i + \alpha \cdot \mu_{global}}{n_c + \alpha}$$

Where:
- $c$ = category
- $\alpha$ = smoothing parameter
- $\mu_{global}$ = global mean of target

In [ ]:
# From-scratch target encoding with smoothing
class TargetEncoder:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.encoding_map = {}
        self.global_mean = None
    
    def fit(self, X, y):
        """Learn encoding from training data"""
        self.global_mean = y.mean()
        
        # Calculate smoothed mean for each category
        for category in X.unique():
            category_mask = (X == category)
            category_target_sum = y[category_mask].sum()
            category_count = category_mask.sum()
            
            # Smoothed mean
            smoothed_mean = (
                (category_target_sum + self.smoothing * self.global_mean) /
                (category_count + self.smoothing)
            )
            
            self.encoding_map[category] = smoothed_mean
        
        return self
    
    def transform(self, X):
        """Apply encoding"""
        return X.map(self.encoding_map).fillna(self.global_mean)
    
    def fit_transform(self, X, y):
        return self.fit(X, y).transform(X)

# Apply to lot_id
encoder = TargetEncoder(smoothing=10.0)
df['lot_id_encoded'] = encoder.fit_transform(df['lot_id'], df['yield'])

print("Target Encoding Results:")
print(df.groupby('lot_id').agg({
    'yield': ['mean', 'count'],
    'lot_id_encoded': 'mean'
}).round(2))

## 6. Feature Hashing for High-Cardinality Categoricals

**Concept:** Hash categorical values into fixed-size feature space.

**Benefits:**
- Handles unlimited categories with fixed memory
- No need to store mapping dictionaries
- Fast encoding

In [ ]:
# Simulate high-cardinality categorical (device serial numbers)
device_ids = [f"DEV_{i:06d}" for i in range(len(df))]
df['device_id'] = device_ids

# Feature hashing
hasher = FeatureHasher(n_features=10, input_type='string')
hashed_features = hasher.transform([[did] for did in df['device_id']])
hashed_df = pd.DataFrame(
    hashed_features.toarray(),
    columns=[f'hash_{i}' for i in range(10)]
)

print(f"Original cardinality: {df['device_id'].nunique()}")
print(f"Hashed features: {hashed_df.shape[1]}")
print(f"\nSample hashed features:")
print(hashed_df.head())

## 7. Automated Feature Engineering with Featuretools

**Concept:** Automatically discover and create features using deep feature synthesis.

In [ ]:
# Note: Featuretools requires installation: pip install featuretools
# For demonstration, we'll show the conceptual approach

print("Automated Feature Engineering Workflow:")
print("\n1. Define entity (table) with index")
print("2. Specify feature primitives (operations)")
print("3. Run deep feature synthesis")
print("4. Select top features by importance")

# Manual example: Create time-based aggregations
# (simulating what Featuretools would do automatically)
df['power'] = df['vdd'] * df['idd']
df['power_density'] = df['power'] / (df['die_x'] + df['die_y'] + 1)
df['thermal_index'] = df['temp'] / df['freq'] * 1000
df['efficiency'] = df['freq'] / df['power']

print("\nAuto-generated features:")
for col in ['power', 'power_density', 'thermal_index', 'efficiency']:
    print(f"  - {col}")

## 8. Complete Feature Engineering Pipeline

In [ ]:
# Combine all engineered features
feature_cols = [
    # Original
    'vdd', 'idd', 'freq', 'temp', 'vth',
    # Interactions
    'vdd_x_idd', 'temp_x_freq', 'vdd_x_vth',
    # Target encoded
    'lot_id_encoded',
    # Auto-generated
    'power', 'power_density', 'thermal_index', 'efficiency'
]

X_full = df_interact[feature_cols]
y = df_interact['yield']

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model_final = Ridge(alpha=1.0)
model_final.fit(X_train_scaled, y_train)
y_pred = model_final.predict(X_test_scaled)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model Performance with Engineered Features:")
print(f"  RMSE: {rmse:.2f}%")
print(f"  R² Score: {r2:.4f}")
print(f"\nTop 5 Most Important Features:")
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': np.abs(model_final.coef_)
}).sort_values('coefficient', ascending=False)
print(feature_importance.head())

## 9. Visualization: Feature Engineering Impact

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Predictions vs Actual
axes[0, 0].scatter(y_test, y_pred, alpha=0.5)
axes[0, 0].plot([70, 100], [70, 100], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Yield (%)')
axes[0, 0].set_ylabel('Predicted Yield (%)')
axes[0, 0].set_title(f'Predictions (R²={r2:.3f})')
axes[0, 0].grid(True)

# 2. Feature Importance
top_features = feature_importance.head(10)
axes[0, 1].barh(range(len(top_features)), top_features['coefficient'])
axes[0, 1].set_yticks(range(len(top_features)))
axes[0, 1].set_yticklabels(top_features['feature'])
axes[0, 1].set_xlabel('|Coefficient|')
axes[0, 1].set_title('Top 10 Feature Importance')
axes[0, 1].invert_yaxis()

# 3. Interaction feature: power vs yield
axes[1, 0].scatter(df_interact['vdd_x_idd'], df_interact['yield'], alpha=0.5)
axes[1, 0].set_xlabel('Power (vdd × idd)')
axes[1, 0].set_ylabel('Yield (%)')
axes[1, 0].set_title('Interaction Feature: Power vs Yield')
axes[1, 0].grid(True)

# 4. Target encoding effect
lot_stats = df_interact.groupby('lot_id')['yield'].agg(['mean', 'std']).reset_index()
axes[1, 1].bar(range(len(lot_stats)), lot_stats['mean'], yerr=lot_stats['std'], capsize=5)
axes[1, 1].set_xticks(range(len(lot_stats)))
axes[1, 1].set_xticklabels(lot_stats['lot_id'])
axes[1, 1].set_ylabel('Yield (%)')
axes[1, 1].set_title('Target Encoding: Lot-Level Patterns')
axes[1, 1].grid(True, axis='y')

plt.tight_layout()
plt.show()

## 10. Project Templates

### Project 1: Wafer-Level Feature Engineering
**Objective:** Create spatial features from die coordinates to predict yield patterns
- Extract radial distance from wafer center
- Calculate neighborhood aggregations (8 surrounding dies)
- Detect edge effects using distance-to-edge features
- Train model with spatial features vs baseline
- **Success Metric:** >15% improvement in yield prediction accuracy

### Project 2: Multi-Parameter Interaction Discovery
**Objective:** Automatically find predictive parameter interactions in STDF data
- Implement exhaustive pairwise interaction search
- Rank interactions by mutual information with yield
- Create top-K interaction features
- Compare with domain-expert selected interactions
- **Success Metric:** Discover 3+ non-obvious interactions with high predictive power

### Project 3: Categorical Feature Optimization
**Objective:** Compare encoding strategies for high-cardinality test program IDs
- Implement one-hot, target, ordinal, and hash encoding
- Measure encoding time and memory usage
- Evaluate model performance with each encoding
- Handle rare categories and missing values
- **Success Metric:** Find optimal encoding with <5% performance loss vs one-hot at 50% memory

### Project 4: Temporal Feature Engineering
**Objective:** Create time-based features from test timestamp data
- Extract hour-of-day, day-of-week patterns
- Calculate time-since-last-test for equipment
- Detect test sequence dependencies
- Build rolling window aggregations
- **Success Metric:** Identify temporal patterns explaining 10%+ yield variance

## 🎓 Key Takeaways

**When to Use Advanced Feature Engineering:**
- Complex, non-linear relationships in data
- Domain knowledge suggests specific interactions
- High-cardinality categoricals (>100 levels)
- Performance matters more than interpretability

**Limitations:**
- ⚠️ Curse of dimensionality with polynomial features
- ⚠️ Target encoding can leak information if not done carefully
- ⚠️ Feature hashing loses interpretability
- ⚠️ Automated methods may create spurious features

**Best Practices:**
1. Start with domain-guided interactions before automation
2. Use regularization (Ridge/Lasso) with many features
3. Validate encoding on separate fold to prevent leakage
4. Monitor feature importance and prune low-value features
5. Document feature engineering logic for production

**Next Steps:**
- Study 104: Model Interpretability to understand engineered features
- Explore AutoML (105) for automated feature discovery
- Apply to real STDF data with >100 test parameters